In [5]:
import numpy as np
import plotly.graph_objs as go
from ipywidgets import widgets
from static_allelic_freq import get_static_plot


In [6]:
# get basic static plot
fig, samples, gnomad_min, gnomad_max, gnomad_freq = get_static_plot()

# parameters
filter_disc_num = 20

/Users/cmesser/PycharmProjects/plotly-interactive/static_allelic_freq.py:65: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/cmesser/PycharmProjects/plotly-interactive/static_allelic_freq.py:67: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Add Widgets

In [ ]:
gnomAD_slider = widgets.FloatSlider(
    value=0,
    min=0,
    max=max(samples[gnomad_freq]),
    step=max(samples[gnomad_freq]) / filter_disc_num,
    disabled=False,
    description='GnomAD Frequency Filter: ',
    readout_format='.5f'
)

gnomAD_nan = widgets.Checkbox(
    value=True,
    description='Allow NaN gnomAD points',
    disabled=False,
    indent=False
)

all_mutations = samples['Variant_Classification'].unique().tolist()
mutation_select = widgets.SelectMultiple(
    options=all_mutations,
    value=all_mutations,
    description='Mutation Types:  ',
    disabled=False
)

ipython_fig = go.FigureWidget(fig)

def response(change):
    filter_list = [m in mutation_select.value and
                   ((g >= gnomAD_slider.value and not np.isnan(g))
                   or (np.isnan(g) and gnomAD_nan.value))
                   for m, g in zip(samples['Variant_Classification'],
                                   samples[gnomad_freq])]
    temp_df = samples[filter_list]
    
    updated_sum_data = temp_df.groupby(level=[0], sort=False).size().to_list()  
    y = temp_df['tumor_f']
    x = temp_df['x_jitter']
    t_count = temp_df['t_ref_count'].values + temp_df['t_alt_count'].values
    with ipython_fig.batch_update():
        ipython_fig.data[0].x = x
        ipython_fig.data[0].y = y
        ipython_fig.data[0].marker['color'] = temp_df[gnomad_freq]
        ipython_fig.data[0].marker['cmin'] = gnomad_min
        ipython_fig.data[0].marker['cmax'] = gnomad_max
        ipython_fig.data[0].customdata = np.stack((temp_df['Hugo_Symbol'].tolist(),
                                                   temp_df['Protein_Change'].tolist(),
                                                   t_count), axis=-1)
        ipython_fig.data[1].z = updated_sum_data
        for x_val, sum in enumerate(updated_sum_data):
            ipython_fig['layout']['annotations'][x_val].update(text=sum)

mutation_select.observe(response, names="value")
gnomAD_slider.observe(response, names="value")
gnomAD_nan.observe(response, names="value")

widgets.VBox([mutation_select, ipython_fig, gnomAD_slider, gnomAD_nan])